# Wikipedia爬蟲練習
## 範例：練習是從Wikipedia中爬取文章。先定義一個搜尋的關鍵字，擷取該關鍵字詞的文章。

In [1]:
import requests
import re
from bs4 import BeautifulSoup

### 先定義一個我們想搜尋的字詞，並將它轉換成UTF-8編碼後的URL

In [55]:
input_keyword = "Flutter"  # 這裡可以自己定義有興趣的關鍵字 (也可以是中文)

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
print("UTF-8 ==> %s: %s" % (input_keyword, utf8_url))     # 擷取中間的編碼結果
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("URL 編碼 (未擷取) ==> %s: %s" % (input_keyword, utf8_url))  
utf_url_path  = utf8_url[2:-1:1]   # 擷取中間的編碼結果 ==> 從 array[2] (0開始) ~ 結尾, 每次間隔 1
print("URL 編碼 ==> %s: %s" % (input_keyword, utf_url_path))   

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf_url_path   
print('\n' + root_keyword_link)

UTF-8 ==> Flutter: B'FLUTTER'
URL 編碼 (未擷取) ==> Flutter: B'FLUTTER'
URL 編碼 ==> Flutter: FLUTTER

/wiki/FLUTTER


### 範例1：送出關鍵字請求後，爬取該關鍵字的文章內容

In [56]:
# 模擬封包的標頭 ===> 測試發現沒有【標題】也可正常抓出內容
headers = {
    'authority': 'zh.wikipedia.org',
    'method': 'GET',
    'path': '/wiki/' + root_keyword_link,
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
    'dnt': '1',
    #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
    'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}    

url = 'https://zh.wikipedia.org' + root_keyword_link  # 組合關鍵字查詢URL
resp = requests.get(url, headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")
content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

#
# 解析回傳資料，並萃取文章內容
#
for paragraph in content:
    print(paragraph.get_text())




維基百科目前還沒有與上述標題相同的條目。請先在維基百科上尋找「FLUTTER」是否已有名稱相近或不同文字寫法的條目。

可能出現此提示的其他原因：



### 範例2：從爬取的文章內容中，擷取出有外部連結的關鍵字。這些關鍵字在文章中是以藍色字體顯示，會連到外部的網頁，並解釋其內容。

In [57]:
for ext_link in content:
    # ^(/wiki/)((?!;)\S)*$ 
    # ==> 以 【/wiki/】 作為開頭
    # ==> 後面接著 > 0 次的字串 ===> 此字串不可以 【;】作為開始或為空白
    a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
    if len(a_tag) > 0:
        for link_string in a_tag:
            a_link = link_string["href"]       # 外部連結的網址
            a_keyword = link_string.get_text()  # 外部連結的中文名稱
            print("外部連結: [%s] %s" % (a_keyword, a_link))

外部連結: [尋找「FLUTTER」] /wiki/Special:%E6%90%9C%E7%B4%A2/FLUTTER


## 作業：接下來定義一個爬蟲函數，這個函數的主要工作為：
### (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)
### (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
### (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

> [python文件讀寫模式 --- r,w,a,r+,w+,a+,rb,wb](https://www.itread01.com/content/1550169482.html)

> [Python split()方法](https://www.runoob.com/python/att-string-split.html)

In [58]:
def WikiArticle(key_word_link, key_word, recursive):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
        local_filepath = 'WikiArticle/' + key_word + '.txt'
        with open(local_filepath, 'w',encoding='utf-8') as f:
            for paragraph in content:
                f.write(paragraph.get_text())

        
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})
        for ext_link in content:
            a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
            if len(a_tag) > 0:
                for link_string in a_tag:
                    a_link = link_string["href"]      
                    if a_link.find('#') > -1:  # 有 Anchor
                        a_link = a_link.split('#',1)[0]   # 以 '#' 為分隔, 分隔成二個
                        
                    a_keyword = link_string.get_text()  
                    if a_link not in external_link_dict:   # 已存在的 key 不重覆寫入
                        external_link_dict[a_link] = a_keyword

                    
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1
            
            for k, v in external_link_dict.items():
                WikiArticle(k, v, recursive)  # 再次呼叫同樣的函數，執行同樣的流程
                

### 執行前個步驟定義好的爬蟲主程式

In [59]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 2

WikiArticle(root_keyword_link, input_keyword, 0)

遞迴層[0] - /wiki/FLUTTER (Flutter)
遞迴層[1] - /wiki/Special:%E6%90%9C%E7%B4%A2/FLUTTER (尋找「FLUTTER」)
遞迴層[2] - /wiki/%E8%B0%B7%E6%AD%8C (Google)
遞迴層[2] - /wiki/%E5%BC%80%E6%BA%90%E8%BD%AF%E4%BB%B6 (開源)
遞迴層[2] - /wiki/Android (Android)
遞迴層[2] - /wiki/IOS (iOS)
遞迴層[2] - /wiki/Google_Fuchsia (Google Fuchsia)
遞迴層[2] - /wiki/Dart (Flutter)
遞迴層[2] - /wiki/%E5%B8%A7%E7%8E%87 (FPS)
遞迴層[2] - /wiki/%E6%B8%B2%E6%9F%93 (彩現)
遞迴層[2] - /wiki/Windows (Windows)
遞迴層[2] - /wiki/MacOS (macOS)
遞迴層[2] - /wiki/Linux (Linux)
遞迴層[2] - /wiki/%E5%8D%B3%E6%99%82%E7%B7%A8%E8%AD%AF (即時編譯)
遞迴層[2] - /wiki/C%2B%2B (C++)
遞迴層[2] - /wiki/Skia_Graphics_Library (Skia圖形庫)
遞迴層[2] - /wiki/%E8%BD%AF%E4%BB%B6%E5%BC%80%E5%8F%91%E5%B7%A5%E5%85%B7%E5%8C%85 (SDK)
遞迴層[2] - /wiki/%E5%BA%94%E7%94%A8%E7%A8%8B%E5%BA%8F%E6%8E%A5%E5%8F%A3 (API)
遞迴層[2] - /wiki/%E7%94%A8%E6%88%B7%E7%95%8C%E9%9D%A2%E8%AE%BE%E8%AE%A1 (使用者介面設計)
遞迴層[2] - /wiki/%E4%B8%8D%E5%8F%AF%E8%AE%8A%E7%89%A9%E4%BB%B6 (不可變)
遞迴層[2] - /wiki/%E8%A8%AD%E8%A8%88%E8%AA%9E%E8%A8%80 (設計